<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Consumer_File%20SQS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3

     |████████████████████████████████| 132 kB 11.5 MB/s 
     |████████████████████████████████| 8.7 MB 47.3 MB/s 
     |████████████████████████████████| 79 kB 11.4 MB/s 
     |████████████████████████████████| 138 kB 62.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import json
import boto3
import datetime
from datetime import date

In [4]:
sqs_send= boto3.client('sqs',  aws_access_key_id='',
aws_secret_access_key='', region_name='')

In [6]:
# Defining Custom Function to Send Message to Main Queue and Deleting it from DLQ 
def sendQueue(sendQueueUrl,messageBody,messageAttributes) :
    # Calling Send Message API
    ret = sqs_send.send_message( QueueUrl=sendQueueUrl, 
                            MessageBody=messageBody,
                            MessageAttributes=messageAttributes) 
    return ret

In [8]:
sourceQueueUrl=""
dlqQueueUrl=""

In [9]:
def consume_message():
  messages = sqs_send.receive_message(QueueUrl=sourceQueueUrl, MaxNumberOfMessages=10
                                    ,WaitTimeSeconds=20,MessageAttributeNames=['All'])
  if 'Messages' in messages:
    for m in messages['Messages']:
      print(m)
      MessageAttributes=m['MessageAttributes']
      messageBody=m['Body']
      try:
        msg_body=json.loads(messageBody)
      except Exception as e:
        print(e)
        response = sendQueue(dlqQueueUrl,messageBody,MessageAttributes)
        sqs_send.delete_message(QueueUrl=sourceQueueUrl, ReceiptHandle=m['ReceiptHandle']) 

In [ ]:
consume_message()